# Alogrithms

## Quadratic Algorithms

### InsertionSort

In [4]:
# Code from L02

def insertion_sort(A):
    for j in range(1, len(A)):
        key = A[j]
        i = j - 1
        while i >= 0 and A[i] > key:
            A[i + 1] = A[i]
            i = i - 1
        A[i + 1] = key
    return A

### BubbleSort

In [5]:
def bubble_sort(array):
    for j in range(0, len(array)-2):
        for k in range(len(array)-1, j, -1):
            if array[k] < array[k-1]:
                array[k], array[k-1] = array[k-1], array[k]
    return array

## Sub-quadratic Algorithms

### MergeSort

In [6]:
# Code from L03

def merge(A, p, q, r):    
    n1 = q - p + 1
    n2 = r - q
    
    L = [0] * n1
    R = [0] * n2

    for i in list(range(n1)):
        L[i] = A[p + i - 1]
    
    for j in list(range(n2)):
        R[j] = A[q + j]
    L.append(float('inf'))
    R.append(float('inf'))

    i = 1 - 1     # Subtract 1 to adjust to Python indexing
    j = 1 - 1     # Subtract 1 to adjust to Python indexing
    
    for k in list(range(p - 1, r)):     # Subtract 1 from q to adjust to Python range object
        if L[i] <= R[j]:
            A[k] = L[i]
            i = i + 1
        else:
            A[k] = R[j]
            j = j + 1
    return A

In [7]:
# See page 34 book for algorithm
def _merge_sort(A, p, r):
    if p < r:
        q = (p + r) // 2
        _merge_sort(A, p, q)
        _merge_sort(A, q + 1, r)
        merge(A, p, q, r)
    return A

In [8]:
def merge_sort(A):
    """
    Algorithm must take only one input parameter to 
    work in benchmarking with the Timer below
    
    Parameters
    ----------
    A : array
        Numbers to be sorted
    """
    p = 1
    r = len(A)
    _merge_sort(A, p, r)
    return A
        

### QuickSort

In [9]:
# Code from L07

def partition(array, low, high):
    """DEFINE PARTITION FOR QUICKSORT"""
    pivot = array[high]
    i = (low - 1)
    for j in range(low, high):
        if array[j] <= pivot:
            i = i + 1
            array[i], array[j] = array[j], array[i]
    array[i + 1], array[high] = array[high], array[i + 1]
    return i + 1


def quick_sort(array, low=0, high=None):
    """Sorts a list using the quicksort algorithm."""
    if high is None:
        high = len(array) - 1
    if low < high:
        part = partition(array, low, high)
        quick_sort(array, low, part - 1)
        quick_sort(array, part + 1, high)
    return array

## Combined Algoriothms

### MergeSort switching to InsertionSort for small data

In [10]:
def combined_sort(A, p=1, n=100):
    """
    Combined algorithm mergesort switching to insertion sort for small data
    
    Parameters
    ----------
    
    A : array 
        Numbers to be sorted
       
    p : int
        Start index. Default=1 for sorting the entire array.
    
    n : int
        Threshold value when the function shifts sorting algorithm
       
    """
    if len(A) < n:
        insertion_sort(A)
    else:
        _merge_sort(A, p, len(A))
    return A

## Built-in sorting functions

## Python 'sort()'

## NumPy 'sort()'

In [3]:
array=[0, -5, 4,8,2,-55, 1,0,-856,94, 9,78,5,-84,568]


### Benchmarking

[-856, -84, -55, -5, 0, 0, 1, 2, 4, 5, 8, 9, 78, 94, 568]

In [ ]:
from benchmarking import run_benchmark

In [25]:
run_benchmark(quick_sort, input_base=2, input_power=11, seed=12)

Minimum time(s) on sorted data of size 1: 4.6549859999868205e-07
Minimum time(s) on sorted data of size 2: 1.4222649999919668e-06
Minimum time(s) on sorted data of size 4: 3.869225999987975e-06
Minimum time(s) on sorted data of size 8: 1.015809000000445e-05
Minimum time(s) on sorted data of size 16: 3.3021019999978305e-05
Minimum time(s) on sorted data of size 32: 0.0001399125499992806
Minimum time(s) on sorted data of size 64: 0.0003925122000000556
Minimum time(s) on sorted data of size 128: 0.0014683160000004136
Minimum time(s) on sorted data of size 256: 0.005551943999998912
Minimum time(s) on sorted data of size 512: 0.02337349000008544
Minimum time(s) on sorted data of size 1024: 0.08877986000006785
Minimum time(s) on sorted data of size 2048: 0.45030320000114443
Minimum time(s) on reversed data of size 1: 5.101451999980781e-07
Minimum time(s) on reversed data of size 2: 1.419414499996492e-06
Minimum time(s) on reversed data of size 4: 3.899638999992021e-06
Minimum time(s) on reve

In [ ]:
from benchmarking import run_benchmark